<a href="https://colab.research.google.com/github/AaronVincent6411/Rock-Paper-Scissor-CNN/blob/main/Rock%2C_Paper%2C_Scissor_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
url = f"https://drive.google.com/uc?id=1w05lAAN30z1Ho7k2731Sv9-THoEPjDe1"
output = "/content/dataset.zip"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1w05lAAN30z1Ho7k2731Sv9-THoEPjDe1
To: /content/dataset.zip
100%|██████████| 474M/474M [00:08<00:00, 55.7MB/s]


'/content/dataset.zip'

In [ ]:
!unzip -qq dataset.zip

In [ ]:
train = '/content/Rock-Paper-Scissors/train'
valid = '/content/Rock-Paper-Scissors/validation'

In [ ]:
train_data = ImageDataGenerator(rescale = 1.0/255, validation_split=0.2)

train_generator = train_data.flow_from_directory(train, batch_size=32, class_mode='categorical', target_size=(300, 300), subset='training')

Found 2016 images belonging to 3 classes.


In [ ]:
image_paths = [os.path.join(valid, filename) for filename in os.listdir(valid)]

preprocessed_images = []
for image_path in image_paths:
    img = Image.open(image_path)
    img = img.resize((300, 300))
    img = np.array(img) / 255.0
    preprocessed_images.append(img)

In [ ]:
validation_data = np.array(preprocessed_images)

In [ ]:
num_images = len(image_paths)
dummy_labels = np.zeros((num_images,))

In [ ]:
validation_generator = ImageDataGenerator().flow(
    x=validation_data,
    y=dummy_labels,
    batch_size=32,
    shuffle=False
)

In [ ]:
model = Sequential([
    Conv2D(64, (5, 5), activation='relu', input_shape=(300, 300, 3)),
    BatchNormalization(),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),

    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 296, 296, 64)      4864      
                                                                 
 batch_normalization (Batch  (None, 296, 296, 64)      256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 296, 296, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 296, 296, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 148, 148, 64)      0         
 D)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 148, 148, 64)      0

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer= Adam(), metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):

    if(logs.get('accuracy') > 0.995):
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(train_generator, epochs=25, verbose=1, steps_per_epoch=len(train_generator), validation_steps=len(validation_generator), callbacks = [callbacks])

Epoch 1/25
 6/63 [=>............................] - ETA: 35s - loss: 117.8672 - accuracy: 0.3438

63/63 [==============================] - 56s 605ms/step - loss: 15.3868 - accuracy: 0.4851
Epoch 2/25
63/63 [==============================] - 38s 601ms/step - loss: 2.8560 - accuracy: 0.9688
Epoch 3/25
63/63 [==============================] - 38s 607ms/step - loss: 1.9678 - accuracy: 0.9945
Epoch 4/25
63/63 [==============================] - 39s 612ms/step - loss: 1.4609 - accuracy: 0.9950


In [ ]:
model.save('DL Course Project.h5')

In [ ]:
model = tf.keras.models.load_model('DL Course Project.h5')

In [ ]:
def detect(test_image_path):
  img = image.load_img(test_image_path, target_size=(300, 300))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0
  class_names = ['Paper', 'Rock', 'Scissor']

  predictions = model.predict(img_array)
  predicted_class_index = np.argmax(predictions[0])
  predicted_class = class_names[predicted_class_index]

  print("Predicted class:", predicted_class)
  print("Predicted probabilities:", predictions[0])


In [ ]:
test_image_path = '/content/Rock-Paper-Scissors/test/scissors/testscissors01-06.png'
detect(test_image_path)
test_image_path = '/content/Rock-Paper-Scissors/test/paper/testpaper02-13.png'
detect(test_image_path)
test_image_path = '/content/Rock-Paper-Scissors/test/rock/testrock03-16.png'
detect(test_image_path)

1/1 [==============================] - 0s 301ms/step
Predicted class: Scissor
Predicted probabilities: [0.26544073 0.23397785 0.5005814 ]
1/1 [==============================] - 0s 20ms/step
Predicted class: Paper
Predicted probabilities: [0.63206744 0.08722274 0.28070986]
1/1 [==============================] - 0s 18ms/step
Predicted class: Rock
Predicted probabilities: [0.22585163 0.6158657  0.15828268]
